In [1]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import cv2
from PIL import Image
from tqdm import tqdm
try:
    from model1 import AgeDetectionResNet, GradCAM, overlay_heatmap
except ImportError as e:
    print(f"ImportError: {e}")
    print("Ensure model.py is in the same directory and contains AgeDetectionResNet, GradCAM, overlay_heatmap")
    exit()

# Set random seed
torch.manual_seed(42)
np.random.seed(42)


ImportError: cannot import name 'AgeDetectionResNet' from 'model1' (c:\Documents\New folder\ag2\model1.py)
Ensure model.py is in the same directory and contains AgeDetectionResNet, GradCAM, overlay_heatmap


In [ ]:

# Verify working directory
print(f"Current working directory: {os.getcwd()}")

# 1. Dataset Preparation
class UTKFaceDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.ages = []
        
        for filename in os.listdir(data_dir):
            if filename.endswith('.jpg'):
                try:
                    age = float(filename.split('_')[0])  # Numerical age
                    self.images.append(os.path.join(data_dir, filename))
                    self.ages.append(age)
                except:
                    print(f"Skipping invalid file: {filename}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            print(f"Error loading image: {img_path}")
            return None
        age = self.ages[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(age, dtype=torch.float32)

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset'
data_dir = r"c:\Users\ponna\Downloads\drive-download-20250525T175035Z-1-001\part1\part1"
try:
    train_dataset = UTKFaceDataset(data_dir, transform=train_transform)
    val_dataset = UTKFaceDataset(data_dir, transform=val_transform)
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

# Split into train/val (80-20)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 2. Model Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AgeDetectionResNet().to(device)
criterion = nn.MSELoss()  # Regression loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Training Loop
num_epochs = 20
best_mae = float('inf')
os.makedirs('weights', exist_ok=True)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, ages in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        if images is None:
            continue
        images, ages = images.to(device), ages.to(device)
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, ages)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}')
    
    # Validation
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for images, ages in val_loader:
            if images is None:
                continue
            images, ages = images.to(device), ages.to(device)
            outputs = model(images).squeeze()
            preds.extend(outputs.cpu().numpy())
            trues.extend(ages.cpu().numpy())
    
    mae = mean_absolute_error(trues, preds)
    mse = mean_squared_error(trues, preds)
    print(f'Validation MAE: {mae:.2f} years, MSE: {mse:.2f}')
    
    # Save best model
    if mae < best_mae:
        best_mae = mae
        torch.save(model.state_dict(), 'weights/resnet18_age_regression.pth')

print(f'Best Validation MAE: {best_mae:.2f} years')

# 4. Predict Ages for Three Images
model.load_state_dict(torch.load('weights/resnet18_age_regression.pth', map_location=device))
model.eval()
grad_cam = GradCAM(model, model.model.layer4[-1])

# Select three random images
image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
if len(image_files) < 3:
    print(f"Error: Found only {len(image_files)} images in {data_dir}. Need at least 3.")
    exit()
np.random.seed(42)
selected_images = np.random.choice(image_files, 3, replace=False)

# Process each image
plt.figure(figsize=(15, 5))
for i, img_file in enumerate(selected_images):
    img_path = os.path.join(data_dir, img_file)
    
    # Extract actual age
    try:
        actual_age = float(img_file.split('_')[0])
    except:
        print(f"Error: Invalid filename format for {img_file}. Skipping.")
        continue
    
    # Load and preprocess image
    image = Image.open(img_path).convert('RGB')
    image_tensor = val_transform(image).unsqueeze(0).to(device)
    image_np = np.array(image.resize((128, 128)))
    
    # Predict age
    with torch.no_grad():
        pred_age = model(image_tensor).squeeze().item()
    
    # Generate Grad-CAM heatmap
    heatmap = grad_cam.generate(image_tensor, target=torch.tensor([1.0], device=device))
    superimposed_img = overlay_heatmap(heatmap, image_np)
    
    # Plot
    plt.subplot(3, 2, 2*i+1)
    plt.imshow(image_np)
    plt.title(f'Original Image\nActual Age: {actual_age:.0f}')
    plt.axis('off')
    
    plt.subplot(3, 2, 2*i+2)
    plt.imshow(superimposed_img)
    plt.title(f'Grad-CAM\nPredicted Age: {pred_age:.1f}')
    plt.axis('off')

plt.tight_layout()
plt.show()

# Print summary
print("\nPrediction Summary:")
for img_file in selected_images:
    try:
        actual_age = float(img_file.split('_')[0])
    except:
        print(f"Error: Invalid filename format for {img_file}. Skipping.")
        continue
    image = Image.open(os.path.join(data_dir, img_file)).convert('RGB')
    image_tensor = val_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        pred_age = model(image_tensor).squeeze().item()
    print(f"Image: {img_file}, Actual Age: {actual_age:.0f}, Predicted Age: {pred_age:.1f}")

Current working directory: c:\Documents\New folder\ag2


NameError: name 'AgeDetectionResNet' is not defined

: 